In [1]:
#import packages
import pandas as pd
import numpy as np
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
import IPython.core.display as ipd
from sklearn.model_selection import KFold
import sys
import time

In [2]:
#set configurations
pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 1000)
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
#dataset
DSL = pd.read_csv('dados/DSL-StrongPasswordData.csv')
DSL.head(5)
DSL = DSL.drop(['sessionIndex', 'rep'], axis=1)

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,DD.e.five,UD.e.five,H.five,DD.five.Shift.r,UD.five.Shift.r,H.Shift.r,DD.Shift.r.o,UD.Shift.r.o,H.o,DD.o.a,UD.o.a,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,0.2212,0.1043,0.1417,1.1885,1.0468,0.1146,1.6055,1.4909,0.1067,0.7590,0.6523,0.1016,0.2136,0.1120,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,0.1357,0.0449,0.0829,1.1970,1.1141,0.0689,0.7822,0.7133,0.1570,0.7877,0.6307,0.1066,0.1684,0.0618,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,0.1542,0.0721,0.0808,1.0408,0.9600,0.0892,0.6203,0.5311,0.1454,0.7195,0.5741,0.1365,0.2931,0.1566,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,0.2038,0.0998,0.0900,1.0556,0.9656,0.0913,1.2564,1.1651,0.1454,0.7550,0.6096,0.0956,0.1530,0.0574,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,0.1589,0.0686,0.0805,0.8629,0.7824,0.0742,0.8955,0.8213,0.1243,0.7632,0.6389,0.0430,0.1975,0.1545,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [4]:
%load_ext autoreload
%autoreload 2

from Biometrics import RecognitionSystem

## DSL

### M2005

In [5]:
inic = time.time()
param_grid = {
    'impostor_rate' : [0.5],
    'model_size' : [20],
    'rate_external_impostor' : [0],
    'decision_threshold' : [0, 1],
    'sampling': ['random']
}

System = RecognitionSystem(method='M2005', adaptive=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid)
fim = time.time() - inic

System.summary(parameters=System.list_params, metrics=System.list_scores)

Split 5 de 5
Usuário 41 de 41


,decision_threshold,impostor_rate,model_size,rate_external_impostor,sampling,Bacc_mean,Bacc_std,FMR_mean,FMR_std,FNMR_mean,FNMR_std
0,0,0.5,20,0,random,0.500013,0.00013,0.0,0.0,0.999974,0.000259
1,1,0.5,20,0,random,0.500000,0.00000,1.0,0.0,0.000000,0.000000


### Euclidian

In [6]:
inic = time.time()
param_grid = {
    'impostor_rate' : [0.5],
    'model_size' : [20],
    'rate_external_impostor' : [0],
    'decision_threshold' : [0, 1],
    'sampling': ['random']
}

System = RecognitionSystem(method='euclidian', adaptive=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid)
fim = time.time() - inic

System.summary(parameters=System.list_params, metrics=System.list_scores)

Split 5 de 5
Usuário 41 de 41


,decision_threshold,impostor_rate,model_size,rate_external_impostor,sampling,Bacc_mean,Bacc_std,FMR_mean,FMR_std,FNMR_mean,FNMR_std
0,0,0.5,20,0,random,0.50427,0.14875,0.664293,0.253015,0.327167,0.30189
1,1,0.5,20,0,random,0.50427,0.14875,0.664293,0.253015,0.327167,0.30189


# Outros - easy

In [7]:
from scipy.io import arff
data, meta = arff.loadarff('dados/easy.arff')
dict_data = dict()
j=0
for feature in meta.names():
    temp_list = list()
    for i in range(len(data)):
        temp_list.append(data[i][j])
    j=j+1
    dict_data[feature] = temp_list
easy = pd.DataFrame(dict_data)
easy = easy.rename(columns={'user_id': 'subject'})

In [8]:
inic = time.time()
param_grid = {
    'impostor_rate' : [0.5],
    'model_size' : [20],
    'rate_external_impostor' : [0],
    'decision_threshold' : [0, 1],
    'sampling': ['random', 'gen_first', 'imp_first']
}

System = RecognitionSystem(method='M2005', adaptive=False)
System.fit(dataset=easy, user_column='subject', param_grid=param_grid)
fim = time.time() - inic

System.summary(parameters=System.list_params, metrics=System.list_scores)

Split 5 de 5
Usuário 44 de 44


,decision_threshold,impostor_rate,model_size,rate_external_impostor,sampling,Bacc_mean,Bacc_std,FMR_mean,FMR_std,FNMR_mean,FNMR_std
0,0,0.5,20,0,random,0.5,0.0,0.0,0.0,1.0,0.0
1,0,0.5,20,0,gen_first,0.5,0.0,0.0,0.0,1.0,0.0
2,0,0.5,20,0,imp_first,0.5,0.0,0.0,0.0,1.0,0.0
3,1,0.5,20,0,random,0.5,0.0,1.0,0.0,0.0,0.0
4,1,0.5,20,0,gen_first,0.5,0.0,1.0,0.0,0.0,0.0
5,1,0.5,20,0,imp_first,0.5,0.0,1.0,0.0,0.0,0.0


In [9]:
inic = time.time()
param_grid = {
    'impostor_rate' : [0.5],
    'model_size' : [20],
    'rate_external_impostor' : [0],
    'decision_threshold' : [0, 1],
    'sampling': ['random']
}

System = RecognitionSystem(method='euclidian', adaptive=False)
System.fit(dataset=easy, user_column='subject', param_grid=param_grid)
fim = time.time() - inic

System.summary(parameters=System.list_params, metrics=System.list_scores)

Split 5 de 5
Usuário 44 de 44


,decision_threshold,impostor_rate,model_size,rate_external_impostor,sampling,Bacc_mean,Bacc_std,FMR_mean,FMR_std,FNMR_mean,FNMR_std
0,0,0.5,20,0,random,0.697294,0.17134,0.480638,0.283983,0.124775,0.257507
1,1,0.5,20,0,random,0.697294,0.17134,0.480638,0.283983,0.124775,0.257507


# Outros - strong

In [10]:
from scipy.io import arff
data, meta = arff.loadarff('dados/strong.arff')
dict_data = dict()
j=0
for feature in meta.names():
    temp_list = list()
    for i in range(len(data)):
        temp_list.append(data[i][j])
    j=j+1
    dict_data[feature] = temp_list
strong = pd.DataFrame(dict_data)
strong = strong.rename(columns = {'user_id' : 'subject'})

In [11]:
inic = time.time()
param_grid = {
    'impostor_rate' : [0.5],
    'model_size' : [20],
    'rate_external_impostor' : [0],
    'decision_threshold' : [0, 1],
    'sampling': ['imp_first']
}

System = RecognitionSystem(method='M2005', adaptive=False)
System.fit(dataset=strong, user_column='subject', param_grid=param_grid)
fim = time.time() - inic

System.summary(parameters=System.list_params, metrics=System.list_scores)

Split 5 de 5
Usuário 44 de 44


,decision_threshold,impostor_rate,model_size,rate_external_impostor,sampling,Bacc_mean,Bacc_std,FMR_mean,FMR_std,FNMR_mean,FNMR_std
0,0,0.5,20,0,imp_first,0.5,0.0,0.0,0.0,1.0,0.0
1,1,0.5,20,0,imp_first,0.5,0.0,1.0,0.0,0.0,0.0


In [12]:
inic = time.time()
param_grid = {
    'impostor_rate' : [0.5],
    'model_size' : [20],
    'rate_external_impostor' : [0],
    'decision_threshold' : [0, 1],
    'sampling': ['gen_first']
}

System = RecognitionSystem(method='euclidian', adaptive=False)
System.fit(dataset=strong, user_column='subject', param_grid=param_grid)
fim = time.time() - inic

System.summary(parameters=System.list_params, metrics=System.list_scores)

Split 5 de 5
Usuário 44 de 44


,decision_threshold,impostor_rate,model_size,rate_external_impostor,sampling,Bacc_mean,Bacc_std,FMR_mean,FMR_std,FNMR_mean,FNMR_std
0,0,0.5,20,0,gen_first,0.701601,0.143455,0.456664,0.247309,0.140134,0.239573
1,1,0.5,20,0,gen_first,0.701601,0.143455,0.456664,0.247309,0.140134,0.239573
